In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
from matplotlib.pyplot import cm
import seaborn as sns
import plotly.graph_objects as go
import plotly.io as pio

In [2]:
df = pd.read_csv('preprocessed_batches\df_hourly_reg.csv', delimiter=',')
print(df)

      hour_from_intime  ABPd   ABPm    ABPs     HR  ICP    RR   SpO2
0                    1   NaN    NaN     NaN    NaN  NaN   NaN    NaN
1                    2   NaN    NaN     NaN    NaN  NaN   NaN    NaN
2                    3   NaN    NaN     NaN    NaN  NaN   NaN    NaN
3                    4  63.2   90.4  135.80  67.60  NaN  17.2  100.0
4                    5  52.5   69.0  100.25  66.75  NaN  18.0  100.0
...                ...   ...    ...     ...    ...  ...   ...    ...
8899                20  67.0   77.0   85.00  81.00  NaN  19.0   94.0
8900                21  64.0   94.0  141.00  81.00  NaN  20.0   97.0
8901                22  60.0   84.0  129.00  81.00  NaN  17.0   99.0
8902                23  58.0   80.0  119.00  77.00  NaN  19.0   98.0
8903                24  74.0  109.0  163.00  80.00  NaN  22.0   99.0

[8904 rows x 8 columns]


In [4]:
import plotly.graph_objects as go


fig = go.Figure()
fig.update_layout(xaxis_title='Blood pressure value analysis (N = 371)', yaxis_title = 'Value (mmHg)')
fig.add_trace(go.Box(y=df['ABPm'], boxpoints='all', name = 'ABPm'))
fig.add_trace(go.Box(y=df['ABPs'], boxpoints='all', name = 'ABPs'))
fig.add_trace(go.Box(y=df['ABPd'], boxpoints='all', name = 'ABPd'))
fig.show()